In [ ]:
from NCA import *
import arc_agi_utils as aau
import cv2
import vft
import os
import matplotlib.pyplot as plt
import numpy as np

from utils import make_path

# Make path for videos

In [ ]:
path_video = "Saved_frames"
make_path(path_video)

# Video making functions

In [ ]:
def write_frame(x, path, frame_number, height, width, chn):
    image_np = x.clone().detach().cpu().permute(0,3,2,1).numpy().clip(0,1)[0,:,:,:3]
    plt.imsave(f"{path}/frame_{frame_number}.png", image_np)

def make_video(path, total_frames, height, width, vid_num = "r"):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(path+'/' +vid_num+'.mp4', fourcc, 15.0, (height, width))
    for frame_number in range(total_frames):
       frame_path = path+f"/frame_{frame_number}.png"
       frame = cv2.imread(frame_path)
       #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
       frame = cv2.flip(frame,1)
       frame = cv2.rotate(frame,cv2.ROTATE_90_COUNTERCLOCKWISE)
       frame = cv2.resize(frame, (height, width), interpolation=cv2.INTER_NEAREST)

       out.write(frame)
    out.release()

# CA settings

In [ ]:
DEVICE = vft.DEVICE
CHANNELS = vft.CHANNELS
BATCH_SIZE = vft.BATCH_SIZE
MASKING = vft.MASKING
GENESIZE = vft.GENESIZE

# Load ARC-AGI

In [ ]:
training_path = "ArcData/data/training"
eval_path = "ArcData/data/evaluation"
(inputs, outputs), (eval_in, eval_out)= aau.import_data(training_path)

max_train = aau.max_n_colors(inputs, outputs)
max_eval = aau.max_n_colors(eval_in, eval_out)
max_colors = max(max_train, max_eval) +1

# Problems to test per model (you can choose your own)

In [ ]:
v3 = {0, 11, 12, 14, 15, 16, 17, 21, 27, 28, 29, 33, 35, 38, 44, 48, 59, 61, 63, 66, 68, 72, 77, 81, 91, 92, 94, 97, 99, 107, 109, 112, 113, 122, 123, 127, 134, 135, 138, 139, 149, 150, 154, 160, 161, 163, 169, 170, 172, 176, 179, 182, 188, 196, 198, 206, 211, 212, 214, 217, 218, 222, 223, 226, 231, 235, 240, 242, 247, 253, 258, 265, 274, 275, 285, 291, 294, 303, 306, 308, 314, 316, 317, 320, 321, 329, 333, 341, 342, 343, 344, 348, 350, 355, 360, 361, 362, 374, 376, 377, 380, 384, 390, 394, 396, 397, 398, 399}



v4 = {7, 12, 140, 24, 154, 155, 159, 162, 41, 169, 172, 49, 185, 69, 205, 208, 209, 213, 89, 94, 222, 103, 107, 116, 118, 120, 249}
v1 = {191, 258, 3, 103, 169, 208, 81, 209, 116, 213, 150, 254, 24, 185, 154, 222, 159}
ca = {7, 136, 140, 142, 24, 159, 162, 41, 169, 172, 49, 185, 63, 191, 69, 71, 205, 209, 213, 89, 94, 223, 103, 231, 235, 118, 120, 254}
test = {0}

# Make Videos 

In [ ]:
for problem in v3:


    mode = "rgb"
    genes = [i for i in range(GENESIZE)]
    nca_in = [aau.pad_to_size([32,32],aau.arc_to_nca_space(max_colors, ip, CHANNELS,GENESIZE, mode=mode, gene_location=genes, is_invis=1)) for id,ip in enumerate(inputs[problem])]
    nca_out = [aau.pad_to_size([32,32], aau.arc_to_nca_space(max_colors, ip, CHANNELS,GENESIZE, mode=mode, gene_location=genes, is_invis=1)) for id,ip in enumerate(outputs[problem])]

    eval_nca_in = [aau.pad_to_size([32,32], aau.arc_to_nca_space(max_colors, ip, CHANNELS,GENESIZE, mode=mode, gene_location=genes, is_invis=1)) for id,ip in enumerate(eval_in[problem])]
    eval_nca_out = [aau.pad_to_size([32,32], aau.arc_to_nca_space(max_colors, ip, CHANNELS,GENESIZE, mode=mode, gene_location=genes, is_invis=1)) for id,ip in enumerate(eval_out[problem])]




    with torch.no_grad():

        nca = EngramNCA_v3(CHANNELS, vft.GENE_HIDDEN_N ,vft.GENE_PROP_HIDDEN_N, GENESIZE)
        nca.load_state_dict(torch.load(f"TrainedARCModels/{type(nca).__name__}/problem_" +str(problem)+".pth"))
        nca = nca.to(DEVICE)

    nca.eval()


    image_np = aau.nca_to_rgb_image(eval_nca_in[0])[:,:,:4]*255
    image_np = cv2.resize(image_np, (10 * image_np.shape[1], 10 * image_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    cv2.imwrite("Saved_frames/Saved_start_and_end/"  + type(nca).__name__ +"problem_start_" + str(problem)+".jpg", image_np)

    image_np = aau.nca_to_rgb_image(eval_nca_out[0])[:, :, :4] * 255
    image_np = cv2.resize(image_np, (10 * image_np.shape[1], 10 * image_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    cv2.imwrite("Saved_frames/Saved_start_and_end/" + type(nca).__name__ + "problem_end_" + str(problem) + ".jpg",
                image_np)


    x = eval_nca_in[0].tile(1,1,1,1)

    for i in range(128):
        x = nca(x, 0.5)
        x = x.detach()
        write_frame(x, path_video, i, 10*x.shape[3],10*x.shape[2], CHANNELS)

    make_video(path_video, 128, 10*x.shape[3],10*x.shape[2], type(nca).__name__ +"problem_" + str(problem) +"padded")